In [ ]:
%matplotlib inline  
#k = 0.6
import random
import math
import sys
import matplotlib.pyplot as plt
from multiprocessing import Pool

def calc_motvation(t):
    if(t == 0):
        return 1
    else:
        return pow(t, motiv_decay)

def calc_attachment(num_edit_history):
    return pow(num_edit_history, attach_increase)

def calc_shorterm_memory(period_last_edit):
    return math.exp(shorterm_mem_decay * period_last_edit) + 1

def gini(list_of_values):
    if(list_of_values.count(list_of_values[0]) == len(list_of_values)):
        if(list_of_values[0] == 0):
            return -1
        else:
            return 0
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2
    return (fair_area - area) / fair_area

def isNotNumber(s):
    try:
        float(s)
        return False
    except ValueError:
        return True

def dot(K, L):
   if len(K) != len(L):
      return 0

   return sum(i[0] * i[1] for i in zip(K, L))

def pearson_1d(x, y):
    """ Pearson product-moment correlation of vectors `x` and `y`

    Parameters
    ----------
    x : array shape (N,)
        One-dimensional array to correlate with `y`
    y : array shape (N,)
        One dimensional array to correlate with `x`

    Returns
    -------
    r_xy : scalar
        Pearson product-moment correlation of vectors `x` and `y`.
    """
    # Mean-center x -> mc_x
    # Mean-center y -> mc_y
    # a : Get sum of products of mc_x, mc_y
    # b : Get sum of products of mc_x on mc_x
    # c : Get sum of products of mc_y on mc_y
    # return a / (sqrt(b) * sqrt(c))
    mc_x = [i - float(sum(x))/float(len(x)) for i in x]
    mc_y = [j - float(sum(y))/float(len(y)) for j in y]
    a = dot(mc_x, mc_y)
    b = dot(mc_x, mc_x)
    c = dot(mc_y, mc_y)
    return a / (math.sqrt(b) * math.sqrt(c))

In [ ]:
born_rate = 0.001 # probability to born a new agnet
motiv_decay = -0.6 # factor to increase the attachment ~ f(n) = t ^ motiv_decay
shorterm_mem_decay = -0.001 # factor to decrease the short-term memory ~ f(t) = exp( shorterm_mem_decay -1)
attach_increase = 1 # factor to increase the attachment ~ f(n) = n ^ attach_increase
remove_prob = 1e-02


#born_rate = float(sys.argv[2]) # probability to born a new agnet
#motiv_decay = float(sys.argv[3]) # factor to increase the attachment ~ f(n) = t ^ motiv_decay
#shorterm_mem_decay = float(sys.argv[4]) # factor to decrease the short-term memory ~ f(t) = exp( shorterm_mem_decay -1)
#attach_increase = float(sys.argv[5]) # factor to increase the attachment ~ f(n) = n ^ attach_increase
#remove_prob = float(sys.argv[6])

max_ensemble = 10
max_edit_step = 50000000

print_res = 10000 # Resolution for printing long-term gini index from the onset
shorterm_print_res = 10000 # Resolution for short-term gini index from the last short-term gini point
point_resolution = 100000

In [ ]:
cur_ensemble = 0
num_editor = []
for j in range(max_edit_step / print_res + 1):  
    num_editor.append(0)
while(cur_ensemble < max_ensemble):
    print cur_ensemble
    cur_edit_index = 1
    insert_date = {}
    last_edit_time = {}
    edit_history = {}
    short_edit_history = {}
    edit_record = []
    editor_list = []
    for i in range(cur_edit_index):
        insert_date[i] = 0
        last_edit_time[i] = 0
        edit_history[i] = 0
    editor_list.append(0)
    edit_step = 0
    while (edit_step < max_edit_step):
        edit_cand_ind = random.randrange(0,len(editor_list))
        edit_cand = editor_list[edit_cand_ind]
        cur_prob = calc_motvation(edit_step - insert_date[edit_cand]) * calc_attachment(edit_history[edit_cand] + 1) * calc_shorterm_memory(edit_step - last_edit_time[edit_cand])
        #print cur_prob
        if(random.random() < cur_prob):
            edit_history[edit_cand] = edit_history[edit_cand] + 1
            if(edit_cand in short_edit_history):
                short_edit_history[edit_cand] = short_edit_history[edit_cand] + 1
            else:
                short_edit_history[edit_cand] = 1
            edit_step = edit_step + 1
            edit_record.append(edit_cand)
            #if(edit_cand < 10):
            if (edit_step % print_res == 0 and edit_step >= print_res):
                #print edit_step, cur_edit_index
                num_editor[edit_step / print_res] = num_editor[edit_step / print_res] + cur_edit_index / float(max_ensemble)
        elif(cur_prob < remove_prob):
            editor_list.pop(edit_cand_ind)
        if(random.random() < born_rate):
            last_edit_time[cur_edit_index] = edit_step
            insert_date[cur_edit_index] = edit_step
            edit_history[cur_edit_index] = 1
            edit_record.append(cur_edit_index)
            editor_list.append(cur_edit_index)
            cur_edit_index = cur_edit_index + 1
            edit_step = edit_step + 1

            if (edit_step % print_res == 0 and edit_step >= print_res):
                #print edit_step, cur_edit_index
                num_editor[edit_step / print_res] = num_editor[edit_step / print_res] + cur_edit_index / float(max_ensemble)
    point_index = 0
    cur_ensemble = cur_ensemble + 1



In [ ]:
import scipy.stats
import numpy as np
print scipy.stats.linregress(np.log(x[1:20]), np.log(num_editor[1:20]))

In [ ]:
import scipy.stats
import numpy as np
print scipy.stats.linregress(np.log(x[20:]), np.log(num_editor[20:]))

In [ ]:
linreg_start = 20
linreg = scipy.stats.linregress(np.log(x[linreg_start:]), np.log(num_editor[linreg_start:]))
linreg_early = scipy.stats.linregress(np.log(x[1:linreg_start]), np.log(num_editor[1:linreg_start]))
print linreg.slope
print len(num_editor)
fn = np.exp(linreg.intercept) * np.power(x, linreg.slope)
fn2 = np.exp(linreg.intercept) * np.power(x, 1)
fn3 = np.exp(linreg_early.intercept) * np.power(x, linreg_early.slope)
plt.xscale('log')
plt.yscale('log')
#plt.title(types[i])
plt.xlabel("$N_e$", fontsize=14)
plt.ylabel("$N_p$", fontsize=14)
x = range(0, max_edit_step+print_res, print_res)
t = range(0, max_edit_step+print_res, print_res)
print len(x)
plt.scatter(x, num_editor, color = "skyblue")
plt.plot(t, fn, '--', color = "red", label = "$y \sim x^{0.80}$")
plt.plot(t, fn3, '-.', color = "green", label = "$y \sim x^{1.30}$")
plt.plot(t, fn2, ':', color = "black", label = "$y \sim x$")
plt.legend(loc=4)
plt.xlim(print_res, max_edit_step+print_res)
plt.ylim(1,)
plt.savefig("./FIG_SUP_025.pdf")
plt.show()
plt.close()